In [1]:
import numpy as np
import pandas as pd

In [2]:
X = np.array([[0, 0],
              [0, 1], 
              [1, 0],
              [1, 1]])

y = np.array([[0],
              [1],
              [1], 
              [0]])

In [3]:
def sigmoid(z):
    return 1/(1 + np.exp(-z))

In [4]:
def MSE(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

In [5]:
class Layer:
    def __init__(self, input_size: int, neuron_count: int):
        self.W = np.random.randn(input_size, neuron_count) * 0.1
        self.b = np.zeros((1, neuron_count))
        
    def forward(self, X: np.ndarray):
        z = X @ self.W + self.b
        a = sigmoid(z)
        return a
        

In [30]:
layer = Layer(input_size=2, neuron_count=3)
layer.forward(X)

array([[0.5       , 0.5       , 0.5       ],
       [0.49666075, 0.51127895, 0.46706556],
       [0.46969529, 0.51866384, 0.4835022 ],
       [0.46636966, 0.52991761, 0.45067496]])

In [23]:
from typing import List


class Network:
    def __init__(self, layers: List[Layer]):
        self.layers = layers
    
    def forward(self, X: np.ndarray):
        for layer in self.layers:
            X = layer.forward(X)
        return X
    
            
layer1 = Layer(input_size=2, neuron_count=3)
layer2 = Layer(input_size=3, neuron_count=1)
layers = [layer1, layer2]
network = Network(layers)

y_pred = network.forward(X)
print(y_pred)
MSE(y, y_pred)

[[0.49962911]
 [0.49925685]
 [0.50005627]
 [0.49968249]]


np.float64(0.24999981854739867)